In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib
import control
from gp_model import gp_model
from helper_fns import *

gp_params = yaml_load('../config/gp_params.yaml')
mpc_params = yaml_load('../config/mpc_params.yaml')
gp_models = gp_model(gp_params, rotation = True)
models, modes = gp_models.load_models(try_reload = False)

test_pt_eu = np.array([1.0, 0.05, 0.2, -1.67, 3.09, -1.8])
test_pt = eulerpose_to_rotpose(test_pt_eu)

Using matplotlib backend: GTK3Agg


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


Loading mode polish with {'data/rail/1.bag', 'data/rail/2.bag', 'data/rail/3.bag'}
Loading ros bag data/rail/1.bag  with 3158 msgs
Loading ros bag data/rail/2.bag  with 6997 msgs
Loading ros bag data/rail/3.bag  with 3767 msgs
Total of 7206 messages after trim
Built model polish with data dim (50, 6)
Mode: polish  | Rotation: False
Max covariance: [22.75]
Min covariance: [22.75]
Max force:      [8.39795959e-08 5.22293989e-08 9.92204751e-08]
Mode: polish  | Rotation: True
Max covariance: [15.8]
Min covariance: [15.47712375]
Max force:      [0.07053365 0.05828072 0.07484313]


/home/hanikevi/git/gp-mpc/src/gp_model.py:248: UserWarning: Requested projection is different from current axis projection, creating new axis with requested projection.
  ax = fig.gca(projection='3d')
/home/hanikevi/git/gp-mpc/src/gp_model.py:248: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax = fig.gca(projection='3d')


GP Likelihood: [-228.60216217324887, -167.52435608027372, -324.5788413732649, -68.74026292364121, -69.54928456265014, -68.5330357394692]


In [45]:
%load_ext autoreload
%autoreload 2
from helper_fns import *
eu1 = np.array([1.178458767867565, 1.5750399390341376, -3.13720775527019])
eu2 = np.array([1.1800346934408186, 1.5722888936312294, 3.1404767855045317])
print((quat_to_rotvec(euler_to_quat(eu1))))
print((quat_to_rotvec(euler_to_quat(eu2))))
print(rotvec_to_rotation(np.array([1.49708619, 0.99597059, 1.48638353])))
print(euler_to_rotation(np.array([1.1800346934408186, 1.5722888936312294, 3.1404767855045317])))
print(rotvec_to_rotation(np.array([2.5239453,  1.69202554, 2.52206465])))
#%timeit models['polish'].predict(test_pt, [])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.393404
1.16646
[-1.49709, -0.995971, -1.48638]
0.392795
1.16713
[-1.49193, -1.00017, -1.49082]

[[0.00567423, 0.00224456, 0.999981], 
 [0.924002, -0.382363, -0.00438484], 
 [0.382346, 0.92401, -0.0042436]]

[[-0.000463246, 0.924619, 0.380892], 
 [0.00180508, -0.380891, 0.924618], 
 [0.999998, 0.00111587, -0.00149257]]

[[-0.000463247, 0.924619, 0.380892], 
 [0.00180508, -0.380891, 0.924618], 
 [0.999998, 0.00111587, -0.00149256]]


In [4]:
%timeit models['polish'].predict_fast(test_pt)

369 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
from gp_dynamics import GPDynamics
gpmodel = GPDynamics(N_p = 6, mpc_params = mpc_params, gp = models['polish'])
dyn = gpmodel.MDS_system()
xk = ca.MX.sym('xk',24)
uk = ca.MX.sym('uk',6)
res = dyn(x=xk, u=uk, init_pose=test_pt, hum_kin_opti = [], imp_params = [] )
dyn_map = dyn.map(3)
res_map = dyn_map(x = ca.MX.sym('xk',24,3), u = ca.MX.sym('uk',6,3), hum_kin_opti = [], imp_params = [])
print(dyn_map)
w = []
w += [ca.horzsplit(res_map['xf'])]
print(ca.reshape(res_map['xf'], 24*3,1))

map3_F_int:(x[24x3],u[6x3],init_pose[6x3],hum_kin_opti[],imp_params[12x3])->(xf[24x3],st_cost[1x3],hum_force_cart[6x3],f_cov[1x3],hum_force_joint[]) Map
vec(map3_F_int(xk, uk, zeros(6x3), 0x0, zeros(12x3)){0})


In [5]:
import casadi as ca
q0 = ca.SX.sym('q0', 6)
from helper_fns import *
x = ca.SX.sym('x',6)
quat_to_rotvec(quat_quat_mult(xyz_to_quat(x[3:]), q0))
#xyz_to_quat(x[:3])

SX(@1=2, @2=cos((x_3/@1)), @3=cos((x_4/@1)), @4=cos((x_5/@1)), @5=(@3*@4), @6=sin((x_3/@1)), @7=sin((x_4/@1)), @8=sin((x_5/@1)), @9=(@7*@8), @10=((@2*@5)+(@6*@9)), @11=((@6*@5)-(@2*@9)), @12=(@7*@4), @13=(@3*@8), @14=((@2*@12)+(@6*@13)), @15=((@2*@13)-(@6*@12)), @16=((((@10*q0_0)-(@11*q0_1))-(@14*q0_2))-(@15*q0_3)), @17=sign(@16), @18=acos((@16*@17)), @19=(@1*@18), [(((((((@10*q0_1)+(@11*q0_0))-(@14*q0_3))+(@15*q0_2))*@17)/sin(@18))*@19), (((((((@10*q0_2)+(@11*q0_3))+(@14*q0_0))-(@15*q0_1))*@17)/sin(@18))*@19), (((((((@10*q0_3)-(@11*q0_2))+(@14*q0_1))+(@15*q0_0))*@17)/sin(@18))*@19)])

In [7]:
# Test if hand optimizing XYZ->QUAT brings something
import casadi as ca
a = ca.SX.sym('a')
b = ca.SX.sym('b')
c = ca.SX.sym('c')
abc = ca.vertcat(a,b,c)

res1 = ca.sin(a)*ca.sin(b)*ca.sin(c)+ca.cos(a)*ca.cos(b)*ca.cos(c)
res2 = 0.25*(ca.sin(a-b-c)+ca.sin(a+b-c)+ca.sin(a-b+c)-ca.sin(a+b+c)
             +ca.cos(a-b-c)+ca.cos(a+b-c)+ca.cos(a-b+c)+ca.cos(a+b+c))

f1 = ca.Function('f1', [a, b, c], [res1])
jf1 = ca.Function('jf1', [a, b, c], [ca.gradient(res1, abc)])
f2 = ca.Function('f2', [a, b, c], [res2])
jf2 = ca.Function('jf2', [a, b, c], [ca.gradient(res2, abc)])

%timeit f1(0.5, -1.1, 0.3)
%timeit f2(0.5, -1.1, 0.3)

%timeit jf1(0.5, -0.1, 0.3)
%timeit jf2(0.5, -0.1, 0.3)

25.9 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
25.7 µs ± 549 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.9 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
25.8 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
